In [1]:
from spacy import displacy
import spacy
spacy.__version__

'3.7.5'

In [2]:
nlp = spacy.load('grc_odycy_joint_trf')

/home/jaredxfeng/sides/nergnt/venvv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read data

In [10]:
import pandas as pd

df = pd.read_csv('acts.csv')

# Inspect

In [16]:
df['Word'].head(20)

0            Τὸν
1            μὲν
2         πρῶτον
3          λόγον
4     ἐποιησάμην
5           περὶ
6         πάντων
7              ὦ
8        Θεόφιλε
9             ὧν
10        ἤρξατο
11             ὁ
12        Ἰησοῦς
13        ποιεῖν
14            τε
15           καὶ
16     διδάσκειν
17          ἄχρι
18            ἧς
19        ἡμέρας
Name: Word, dtype: object

In [17]:
df['Proper Noun'].value_counts()

Proper Noun
TRUE,PERSON    374
TRUE,GPE       177
TRUE,DIVINE    157
TRUE,GROUP     141
TRUE,TITLE     117
TRUE,ORG        22
TRUE,LOC        21
TRUE,DEITY      11
TRUE,EVENT       8
TRUE,BOOK        7
TRUE,LANG        2
TRUE,TIME        1
Name: count, dtype: int64

In [18]:
len(df)

10038

# Split train vs. test

Take the first 7 chapters as training data

In [20]:
df['Verse'].astype(int)

ValueError: invalid literal for int() with base 10: '050214"'

In [ ]:
df_train = df

### Concat all words with white space separations

In [67]:
doc = nlp(' '.join(input))

In [68]:
for i, token in enumerate(doc):
    print(token.lemma_, token.is_stop, token.pos_, token.morph, token.dep_, token.head)
    if i == 20:
        break

ὁ True DET Case=Acc|Definite=Def|Gender=Masc|Number=Sing|PronType=Dem det ΛΌΓΟΝ
μέν True ADV  discourse ἘΠΟΙΗΣΆΜΗΝ
ΠΡΩ͂ΤΟΝ False ADJ Case=Acc|Gender=Masc|Number=Sing nmod ΛΌΓΟΝ
λόγος False NOUN Case=Acc|Gender=Masc|Number=Sing obj ἘΠΟΙΗΣΆΜΗΝ
ποιέω False VERB Aspect=Perf|Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin|Voice=Mid ROOT ἘΠΟΙΗΣΆΜΗΝ
περί True ADP  case ΠΆΝΤΩΝ
πᾶς False ADJ Case=Gen|Gender=Neut|Number=Plur obl ἘΠΟΙΗΣΆΜΗΝ
ὦ True INTJ  discourse ΘΕΌΦΙΛΕ
θεόφιλος False PROPN Case=Voc|Gender=Masc|Number=Sing vocative ἘΠΟΙΗΣΆΜΗΝ
ὅς True PRON Case=Gen|Gender=Masc|Number=Plur|PronType=Rel obj ΠΟΙΕΙ͂Ν
ἄρχω False VERB Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin|Voice=Mid acl ΛΌΓΟΝ
ὁ True DET Case=Nom|Definite=Def|Gender=Masc|Number=Sing|PronType=Dem det ἸΗΣΟΥ͂Σ
ἸΗΣΟ(Υ)͂Σ False PROPN Case=Nom|Gender=Masc|Number=Sing nsubj ἬΡΞΑΤΟ
ΠΟΙΕΙ͂Ν False VERB Tense=Pres|VerbForm=Inf|Voice=Act xcomp ἬΡΞΑΤΟ
τε True CCONJ  cc ΠΟΙΕΙ͂Ν
καί True CCONJ  cc ΠΟΙΕΙ͂Ν
διδάσκω False

In [69]:
nlp.pipe_names

['transformer',
 'tagger',
 'morphologizer',
 'parser',
 'trainable_lemmatizer',
 'frequency_lemmatizer']

In [70]:
nlp.add_pipe("ner")

In [71]:
nlp.pipe_names

['transformer',
 'tagger',
 'morphologizer',
 'parser',
 'trainable_lemmatizer',
 'frequency_lemmatizer',
 'ner']